In [7]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.font_manager import FontProperties

from utils.models import *

try:
    myfont = FontProperties(fname=r"/System/Library/Fonts/PingFang.ttc")
    sns.set(style="whitegrid", font=myfont.get_name())
except Exception as e:
    print(e)

plt.rcParams['font.sans-serif'] = ['Microsoft YaHei']
plt.rcParams['axes.unicode_minus'] = False

dataA2 = pd.read_csv("../Version3/Data/A2.csv", low_memory=False)
dataA1 = pd.read_csv("../Version3/Data/A1.csv")

[Errno 2] No such file or directory: 'C:\\System\\Library\\Fonts\\PingFang.ttc'


In [23]:
car_0 = pd.read_csv(os.path.join("../Version3/Data/CarData/full_0.csv"), encoding='utf-8')
car_1 = pd.read_csv(os.path.join("../Version3/Data/CarData/full_1.csv"), encoding='utf-8')
car_2 = pd.read_csv(os.path.join("../Version3/Data/CarData/full_2.csv"), encoding='utf-8')
car_out = pd.read_csv(os.path.join("../Version3/Data/CarData/full_out.csv"), encoding='utf-8')
car_overlap = pd.read_csv(os.path.join("../Version3/Data/CarData/overlap_data.csv"), encoding='utf-8')

motor_0 = pd.read_csv(os.path.join("../Version3/Data/MotorData/full_0.csv"), encoding='utf-8')
motor_1 = pd.read_csv(os.path.join("../Version3/Data/MotorData/full_1.csv"), encoding='utf-8')
motor_out = pd.read_csv(os.path.join("../Version3/Data/MotorData/full_out.csv"), encoding='utf-8')
motor_overlap = pd.read_csv(os.path.join("../Version3/Data/MotorData/overlap_data.csv"), encoding='utf-8')

pass_0 = pd.read_csv(os.path.join("../Version3/Data/PassData/full_0.csv"), encoding='utf-8')
pass_1 = pd.read_csv(os.path.join("../Version3/Data/PassData/full_1.csv"), encoding='utf-8')
pass_out = pd.read_csv(os.path.join("../Version3/Data/PassData/full_out.csv"), encoding='utf-8')
pass_overlap = pd.read_csv(os.path.join("../Version3/Data/PassData/overlap_data.csv"), encoding='utf-8')

full_data = pd.read_csv(os.path.join("../Version3/Data/FullData/full.csv"), encoding='utf-8')

### 這是由拓樸得出來的特徵，可新增於拓樸訓練，但不增加在full_data

In [25]:
car_out['type'] = 'out'
car_overlap['type'] = 'overlap'
motor_out['type'] = 'out'
motor_overlap['type'] = 'overlap'
pass_out['type'] = 'out'
pass_overlap['type'] = 'overlap'

car_out_overlap = pd.concat([car_out, car_overlap])
motor_out_overlap = pd.concat([motor_out, motor_overlap])
pass_out_overlap = pd.concat([pass_out, pass_overlap])

### 行人資料需要特別處理，因為他們沒有以下特徵

In [26]:
pass_0['行動電話或電腦或其他相類功能裝置名稱'] = '非駕駛人'
pass_0['當事者區分-類別-大類別名稱-車種'] = '人'
pass_1['行動電話或電腦或其他相類功能裝置名稱'] = '非駕駛人'
pass_1['當事者區分-類別-大類別名稱-車種'] = '人'
pass_out['行動電話或電腦或其他相類功能裝置名稱'] = '非駕駛人'
pass_out['當事者區分-類別-大類別名稱-車種'] = '人'
pass_overlap['行動電話或電腦或其他相類功能裝置名稱'] = '非駕駛人'
pass_overlap['當事者區分-類別-大類別名稱-車種'] = '人'
pass_out_overlap['行動電話或電腦或其他相類功能裝置名稱'] = '非駕駛人'
pass_out_overlap['當事者區分-類別-大類別名稱-車種'] = '人'

In [27]:
select_lst = [
    '天候名稱', '光線名稱', 
    '道路類別-第1當事者-名稱', '速限-第1當事者', 
    
    # 路面狀況
    '路面狀況-路面鋪裝名稱', '路面狀況-路面狀態名稱', '路面狀況-路面缺陷名稱',
    '道路障礙-障礙物名稱', '道路障礙-視距品質名稱', '道路障礙-視距名稱',
    # 號誌
    '號誌-號誌種類名稱', '號誌-號誌動作名稱',
    # 車道
    '車道劃分設施-分道設施-快車道或一般車道間名稱', '車道劃分設施-分道設施-快慢車道間名稱', '車道劃分設施-分道設施-路面邊線名稱',
    # 當事人
    '當事者屬-性-別名稱', '當事者事故發生時年齡',
    '保護裝備名稱', '行動電話或電腦或其他相類功能裝置名稱',
    '肇事逃逸類別名稱-是否肇逃',

    # 大類別
    '道路型態大類別名稱', '事故位置大類別名稱',
    '車道劃分設施-分向設施大類別名稱',
    '事故類型及型態大類別名稱', '當事者區分-類別-大類別名稱-車種',
    '車輛撞擊部位大類別名稱-其他',
    '肇因研判大類別名稱-主要',

    # 子類別
    '道路型態子類別名稱', '事故位置子類別名稱', '事故類型及型態子類別名稱', '肇因研判子類別名稱-主要',
    '當事者區分-類別-子類別名稱-車種', '當事者行動狀態子類別名稱', '車輛撞擊部位子類別名稱-最初',
    '車輛撞擊部位子類別名稱-其他', '肇因研判子類別名稱-個別',
    
    '死亡'
]

# select data
car_0 = car_0[select_lst]
car_1 = car_1[select_lst]
car_2 = car_2[select_lst]
motor_0 = motor_0[select_lst]
motor_1 = motor_1[select_lst]
pass_0 = pass_0[select_lst]
pass_1 = pass_1[select_lst]

full_data = full_data[select_lst]

select_lst.append('type')
motor_out_overlap = motor_out_overlap[select_lst]
car_out_overlap = car_out_overlap[select_lst]
pass_out_overlap = pass_out_overlap[select_lst]

## 唯一值檢查
第一區塊是完整模型的資料<br/>
第二區塊是多個模型個別進行模型，所以這裡需要量化後自動drop

In [28]:
print('詳細群體')
for data in [car_0, car_1, car_2, car_out_overlap, motor_0, motor_1, motor_out_overlap, pass_0, pass_1, pass_out_overlap, full_data]:
    columns_to_drop = []
    for column in data.columns:
        if data[column].nunique() == 1:
            columns_to_drop.append(column)
    print(columns_to_drop)
    data.drop(columns=columns_to_drop, inplace=True)

詳細群體
['道路障礙-視距品質名稱', '道路障礙-視距名稱']
['道路障礙-視距品質名稱', '道路障礙-視距名稱']
['道路障礙-視距品質名稱', '道路障礙-視距名稱']
[]
['道路障礙-視距品質名稱', '道路障礙-視距名稱']
['道路障礙-視距品質名稱', '道路障礙-視距名稱']
[]
['路面狀況-路面缺陷名稱', '道路障礙-視距品質名稱', '道路障礙-視距名稱', '號誌-號誌動作名稱', '行動電話或電腦或其他相類功能裝置名稱', '事故類型及型態大類別名稱', '當事者區分-類別-大類別名稱-車種']
['道路障礙-視距品質名稱', '道路障礙-視距名稱', '行動電話或電腦或其他相類功能裝置名稱', '當事者區分-類別-大類別名稱-車種']
['行動電話或電腦或其他相類功能裝置名稱', '肇事逃逸類別名稱-是否肇逃', '當事者區分-類別-大類別名稱-車種']
[]


In [29]:
from sklearn.ensemble import RandomForestClassifier

def drop_low_importance_features(data, threshold=0.01):
    # One-hot 編碼資料
    X = pd.get_dummies(data.drop(columns=['死亡']))
    y = data['死亡']

    # 訓練 RandomForest 模型
    model = RandomForestClassifier()
    model.fit(X, y)

    # 計算特徵重要性
    feature_importances = pd.Series(model.feature_importances_, index=X.columns).sort_values(ascending=False)

    # 篩選重要性高於 threshold 的 dummy 特徵
    selected_dummy_features = feature_importances[feature_importances > threshold].index.tolist()

    # 找回原始的類別特徵名稱
    original_features = data.drop(columns=['死亡']).columns
    selected_features = set()

    for feature in original_features:
        # 如果原始特徵的任何 dummy 特徵被選中，就保留原始特徵
        if any(dummy.startswith(f"{feature}_") for dummy in selected_dummy_features) or feature in selected_dummy_features:
            selected_features.add(feature)

    # 回到原始資料集中的選擇特徵
    reduced_data = data[list(selected_features) + ['死亡']]
    print(f"原始特徵數: {data.shape[1]}, 保留特徵數: {len(selected_features)}")
    return reduced_data

car0_reduced = drop_low_importance_features(car_0)
car1_reduced = drop_low_importance_features(car_1)
car2_reduced = drop_low_importance_features(car_2)
car_out_overlap_reduced = drop_low_importance_features(car_out_overlap)
motor0_reduced = drop_low_importance_features(motor_0)
motor1_reduced = drop_low_importance_features(motor_1)
motor_out_overlap_reduced = drop_low_importance_features(motor_out_overlap)
pass0_reduced = drop_low_importance_features(pass_0)
pass1_reduced = drop_low_importance_features(pass_1)
pass_out_overlap_reduced = drop_low_importance_features(pass_out_overlap)
full_data_reduced = drop_low_importance_features(full_data)

原始特徵數: 35, 保留特徵數: 14
原始特徵數: 35, 保留特徵數: 17
原始特徵數: 35, 保留特徵數: 16
原始特徵數: 38, 保留特徵數: 7
原始特徵數: 35, 保留特徵數: 15
原始特徵數: 35, 保留特徵數: 15
原始特徵數: 38, 保留特徵數: 17
原始特徵數: 30, 保留特徵數: 20
原始特徵數: 33, 保留特徵數: 17
原始特徵數: 35, 保留特徵數: 19
原始特徵數: 37, 保留特徵數: 11


In [30]:
car_0_dummy = pd.get_dummies(car0_reduced)
car_1_dummy = pd.get_dummies(car1_reduced)
car_2_dummy = pd.get_dummies(car2_reduced)
car_out_overlap_dummy = pd.get_dummies(car_out_overlap_reduced)
motor_0_dummy = pd.get_dummies(motor0_reduced)
motor_1_dummy = pd.get_dummies(motor1_reduced)
motor_out_overlap_dummy = pd.get_dummies(motor_out_overlap_reduced)
pass_0_dummy = pd.get_dummies(pass0_reduced)
pass_1_dummy = pd.get_dummies(pass1_reduced)
pass_out_overlap = pd.get_dummies(pass_out_overlap_reduced)

full_data_dummy = pd.get_dummies(full_data_reduced)

In [31]:
car_0_X, car_0_y = get_train_test_data(car_0_dummy)
car_1_X, car_1_y = get_train_test_data(car_1_dummy)
car_2_X, car_2_y = get_train_test_data(car_2_dummy)
car_out_overlap_X, car_out_overlap_y = get_train_test_data(car_out_overlap_dummy)
motor_0_X, motor_0_y = get_train_test_data(motor_0_dummy)
motor_1_X, motor_1_y = get_train_test_data(motor_1_dummy)
motor_out_overlap_X, motor_out_overlap_y = get_train_test_data(motor_out_overlap_dummy)
pass_0_X, pass_0_y = get_train_test_data(pass_0_dummy)
pass_1_X, pass_1_y = get_train_test_data(pass_1_dummy)
pass_out_overlap_X, pass_out_overlap_y = get_train_test_data(pass_out_overlap)

full_data_X, full_data_y = get_train_test_data(full_data_dummy)

In [32]:
print(car_0_y.value_counts())
print(car_1_y.value_counts())
print(car_2_y.value_counts())
print(car_out_overlap_y.value_counts())
print(motor_0_y.value_counts())
print(motor_1_y.value_counts())
print(motor_out_overlap_y.value_counts())
print(pass_0_y.value_counts())
print(pass_1_y.value_counts())
print(pass_out_overlap_y.value_counts())

print(full_data_y.value_counts())

0    57472
1      285
Name: y, dtype: int64
0    43045
1      175
Name: y, dtype: int64
0    2084
1      35
Name: y, dtype: int64
0    8772
1      88
Name: y, dtype: int64
0    69156
1      318
Name: y, dtype: int64
0    32699
1       87
Name: y, dtype: int64
0    5353
1      18
Name: y, dtype: int64
0    529
1     11
Name: y, dtype: int64
0    2373
1      33
Name: y, dtype: int64
0    183
1     14
Name: y, dtype: int64
0    218238
1       965
Name: y, dtype: int64


In [33]:
print(car_0_X.shape)
print(car_1_X.shape)
print(car_2_X.shape)
print(car_out_overlap_X.shape)
print(motor_0_X.shape)
print(motor_1_X.shape)
print(motor_out_overlap_X.shape)
print(pass_0_X.shape)
print(pass_1_X.shape)
print(pass_out_overlap_X.shape)
print(full_data_X.shape)

(57757, 106)
(43220, 162)
(2119, 248)
(8860, 70)
(69474, 122)
(32786, 96)
(5371, 123)
(540, 100)
(2406, 117)
(197, 119)
(219203, 78)


In [34]:
car_0_X.shape[0] + car_1_X.shape[0] + car_2_X.shape[0] + car_out_overlap_X.shape[0] + motor_0_X.shape[0] + motor_1_X.shape[0] + motor_out_overlap_X.shape[0] + pass_0_X.shape[0] + pass_1_X.shape[0] + pass_out_overlap_X.shape[0], full_data_X.shape[0]

(222730, 219203)

## 全部大類別資料kfold計算到motor0開始

In [35]:
def xgboost_cm_gridsearch(X, y, random_state=42, n_jobs=12):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=random_state)

    smote = SMOTE(random_state=random_state, k_neighbors=3)
    enn = EditedNearestNeighbours(n_neighbors=3)
    smote_enn = SMOTEENN(smote=smote, enn=enn, random_state=random_state)
    X_resampled_train, y_resampled_train = smote_enn.fit_resample(X_train, y_train)

    min_class_count = min(sum(y_test == 0), sum(y_test == 1))
    rus_test = RandomUnderSampler(sampling_strategy={0: min_class_count, 1: min_class_count}, random_state=random_state)
    X_resampled_test, y_resampled_test = rus_test.fit_resample(X_test, y_test)

    model = XGBClassifier(eval_metric='logloss', random_state=random_state)
    parameters = {
        'n_estimators': [50, 100, 200, 400], # 樹的数量
        'max_depth': [3, 5, 7, 9],
        'learning_rate': [0.01, 0.1, 0.2],
        'colsample_bytree': [0.8, 1.0],  # 每棵樹使用的特征采样比例
    }
    grid_search = GridSearchCV(model, parameters, cv=5, scoring='accuracy', n_jobs=n_jobs)
    grid_search.fit(X_resampled_train, y_resampled_train)
    best_model = grid_search.best_estimator_

    print("Best parameters found by GridSearchCV:", grid_search.best_params_)

    y_proba = best_model.predict_proba(X_resampled_test)[:, 1]

    return y_resampled_test, y_proba, np.arange(len(y_resampled_test))


In [36]:
import pickle
import time
import gc

models = [
    # ("pass_0", pass_0_X, pass_0_y),
    # ("pass_1", pass_1_X, pass_1_y),
    # ("pass_out_overlap", pass_out_overlap_X, pass_out_overlap_y),
    # ("car_0", car_0_X, car_0_y),
    # ("car_1", car_1_X, car_1_y),
    # ("car_2", car_2_X, car_2_y),
    # ("car_out_overlap", car_out_overlap_X, car_out_overlap_y),
    # ("motor_0", motor_0_X, motor_0_y),
    # ("motor_1", motor_1_X, motor_1_y),
    # ("motor_out_overlap", motor_out_overlap_X, motor_out_overlap_y),
    ("full_data", full_data_X, full_data_y),
]

# XGBoost no fold
for name, X, y in models:
    print(f'{name} xgboost start')
    start_time = time.time()
    y_xgb, decision_scores_xgb, indices_xgb = xgboost_cm_gridsearch(X.astype(float), y)
    end_time = time.time()
    elapsed_time = end_time - start_time
    with open(f"../Models/ModelPerformanceNofold/xgboost/{name}.pkl", "wb") as f:
        pickle.dump({
            'y': y_xgb,
            'decision_scores': decision_scores_xgb,
            'indices': indices_xgb,
            'elapsed_time': elapsed_time
        }, f)
    print(f'{name} xgboost done in {elapsed_time:.2f} seconds')
    del X, y, y_xgb, decision_scores_xgb
    gc.collect()

# Logistic no fold
for name, X, y in models:
    print(f'{name} logistic start')
    start_time = time.time()
    # y_log, decision_scores_log, indices_log = logistic_cm_kfold(X.astype(float), y)
    y_log, decision_scores_log, indices_log = logistic_cm_gridsearch(X.astype(float), y)
    end_time = time.time()
    elapsed_time = end_time - start_time
    with open(f"../Models/ModelPerformanceNofold/logistic/{name}.pkl", "wb") as f:
        pickle.dump({
            'y': y_log,
            'decision_scores': decision_scores_log,
            'indices': indices_log,
            'elapsed_time': elapsed_time
        }, f)
    print(f'{name} logistic done in {elapsed_time:.2f} seconds')
    del X, y, y_log, decision_scores_log
    gc.collect()

# SVC no fold
for name, X, y in models:
    print(f'{name} svc start')
    start_time = time.time()
    y_svc, decision_scores_svc, indices_svc = linear_svc_cm_gridsearch(X.astype(float), y)
    end_time = time.time()
    elapsed_time = end_time - start_time
    with open(f"../Models/ModelPerformanceNofold/svc/{name}.pkl", "wb") as f:
        pickle.dump({
            'y': y_svc,
            'decision_scores': decision_scores_svc,
            'indices': indices_svc,
            'elapsed_time': elapsed_time
        }, f)
    print(f'{name} svc done in {elapsed_time:.2f} seconds')
    del X, y, y_svc, decision_scores_svc
    gc.collect()

full_data xgboost start
Best parameters found by GridSearchCV: {'colsample_bytree': 1.0, 'learning_rate': 0.2, 'max_depth': 9, 'n_estimators': 400}
full_data xgboost done in 1159.79 seconds
full_data logistic start
Best parameters found by GridSearchCV: {'C': 100, 'penalty': 'l2'}
full_data logistic done in 683.69 seconds
full_data svc start
Best parameters found by GridSearchCV: {'C': 10, 'loss': 'squared_hinge'}
full_data svc done in 385.17 seconds
